In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

In [2]:
import pandas as pd

In [3]:
datapath="../../datasets/OpenSecrets/Yang_Ruan_Data_20240304/"
datapath_og="../../datasets/OpenSecrets/Yang_Ruan_Data_20240304_original/"

In [15]:
valid = pd.read_csv(f"{datapath}lob_indus.csv", encoding="utf8")

In [16]:
valid.loc[8061]

client     Alaka’i Defense Systems
sub        Alaka’i Defense Systems
total                      20000.0
year                          2023
type                            q3
catcode                      D4000
Name: 8061, dtype: object

In [31]:
valid.head()

,Unnamed: 0,client,sub,total,year,type,catcode
0,0,(ISC)2,(ISC)2,15000.0,2018,q4,Y4000
1,1,(ISC)2,(ISC)2,40000.0,2021,q4,Y4000
2,2,(ISC)2,(ISC)2,30000.0,2021,q3,Y4000
3,3,"1,000 Days","1,000 Days",20000.0,2018,q2,Y4000
4,4,"1,000 Days","1,000 Days",30000.0,2018,q1,Y4000


In [15]:
'Alamance County Sheriff''s Office'

'Alamance County Sheriffs Office'

In [8]:
raw = pd.read_csv(f"{datapath_og}Yang_Ruan_lob_indus.txt", quotechar="'", encoding="windows-1252")

In [10]:
raw.head()

,client,sub,total,year,type,catcode
0,(ISC)2,(ISC)2,15000.0,2018,q4,Y4000
1,(ISC)2,(ISC)2,40000.0,2021,q4,Y4000
2,(ISC)2,(ISC)2,30000.0,2021,q3,Y4000
3,"1,000 Days","1,000 Days",20000.0,2018,q2,Y4000
4,"1,000 Days","1,000 Days",30000.0,2018,q1,Y4000


In [9]:
raw.loc[8061]

client     Alaka’i Defense Systems
sub        Alaka’i Defense Systems
total                      20000.0
year                          2023
type                            q3
catcode                      D4000
Name: 8061, dtype: object

In [14]:
raw.to_csv(f"{datapath}lob_indus.csv", encoding='utf8', index=False)

In [ ]:
table = "agency"

In [10]:
raw = pd.read_csv(f"{datapath}Yang_Ruan_lob_agency.txt")